<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

## Practice 1-Sample T-tests

One Sample t-tests determine whether or not a sample mean is statistically different from some known (or hypothesized) population mean. 

### 1) Load the Data
- Use the [automobile dataset](https://archive.ics.uci.edu/ml/datasets/Automobile)
- Fix the column headers
- Make sure NaNs are used to indicate missing values

Feel free to add code cells and text cells as needed throughout the assignment.

In [156]:
### YOUR WORK HERE

# imports

import numpy as np
import pandas as pd
from scipy import stats

# .csv load

auto_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data')

In [157]:
# column headers

auto_data.columns = ['symboling', 
                     'normalized-losses',
                     'make',
                     'fuel-type',
                     'aspiration',
                     'num-of-doors',
                     'body-style',
                     'drive-wheels',
                     'engine_location',
                     'wheel-base',
                     'length',
                     'width',
                     'height',
                     'curb-weight',
                     'engine-type',
                     'num-of-cylinders',
                     'engine-size',
                     'fuel-system',
                     'bore',
                     'stroke',
                     'compression-ratio',
                     'horsepower',
                     'peak-rpm',
                     'city-mpg',
                     'highway-mpg',
                     'price']

# using head() to look for missing values

auto_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine_location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,?,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [158]:
# checking to see how many '?' there are

auto_data.isin(['?']).sum()

# turning '?'s to NaNs

auto_data_nanfix = auto_data.replace('?', np.NaN)

auto_data_nanfix.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine_location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


### 2) Pretend that this dataset represents the cars at a used car lot in your local town. 

- Use df.sample() to pick a random sample of 10 cars. Note that because this sample is random we are going to set the `random_state` so that all of us in the class get the same random sample. Please set your random state to `30` when using `df.sample()`

Is your sample reflective of the population value in regards to highway-mpg? Find the mean for `highway-mpg` for the entire dataset and compare your sample mean estimate to that value. Can you say that they are different? 

In [159]:
### YOUR WORK HERE

# Creating sample

auto_data_nanfix.sample(10, random_state=30)

# Printing sample mean and dataset mean of highway-mpg

hwympg_mean = auto_data_nanfix['highway-mpg'].mean()
hwympg_sample_mean = auto_data_nanfix['highway-mpg'].sample(10, random_state=30).mean()

# These means are pretty close, but still different

print(hwympg_mean, hwympg_sample_mean)

# Dropping NaNs and recalculating, creates a larger discrepancy between means

auto_data_nanfix = auto_data_nanfix.dropna()
hwympg_mean = auto_data_nanfix['highway-mpg'].mean()
hwympg_sample_mean = auto_data_nanfix['highway-mpg'].sample(10, random_state=30).mean()

# These means are pretty close, but still different

print(hwympg_mean, hwympg_sample_mean)

# Running a t-test

auto_ttest_sample = auto_data_nanfix.sample(10, random_state=30)

stats.ttest_ind(auto_data_nanfix['highway-mpg'], 
                auto_ttest_sample['highway-mpg'], 
                nan_policy='omit')

# Confidence value of 76% - not statistically significant
# Confidence value of 27% when NaNs aren't dropped - statistically insignificant
# I commented out my dropna() to get the second confidence value

30.769607843137255 30.3
32.081761006289305 29.5


Ttest_indResult(statistic=1.1777237560964617, pvalue=0.24058237159349663)

The salesman says the cars he sells typically have a fuel efficiency of about 35 miles per gallon on the highway. You want to verify his claim but can only test 10 cars. Using your sample of 10, test his claim and report your results.

In [161]:
### YOUR WORK HERE

# T-test of sample against 35mpg-hwy

stats.ttest_1samp(auto_ttest_sample['highway-mpg'], 35)

# Confidence value of 88% is not quite statistically significant
# His claim is dubious, but we can't disprove it with this sample size
# We fail to reject his hypothesis

Ttest_1sampResult(statistic=-1.6853413858487127, pvalue=0.12620426076756258)

Imagine that you now have the capacity to test 100 cars on the lot. Using the same random state of `30`, what would you conclude when sampling 100 cars?

In [186]:
### YOUR WORK HERE

# New sample

auto_ttest_sample100 = auto_data_nanfix.sample(100, random_state=30)

stats.ttest_1samp(auto_ttest_sample100['highway-mpg'], 35)

# Confidence value of nearly 100% is very statistically significant
# His claim is almost completely unverified
# The confidence value plummets to 0 if we change the 2nd parameter to 31.59
# We reject his hypothesis

Ttest_1sampResult(statistic=0.0, pvalue=1.0)

Why might these two t-tests using the same dataset lead to different conclusions about the salesman's claim?

A larger sample size results in more accurate and more robust results.

## Practice 2-Sample T-tests

Two Sample t-tests determine whether or not two sample means are statistically different from each other. 

This portion of your assignment requires you to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). 


### 1) Load the data

The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

- Read the dataset in from UCI, you'll need to provide a list of column headers
- Encode "yes" votes as 1 and "no" votes as 0. (You can use `df.replace()` to swap out these values)
- Use dataframe filtering to split the dataframe into two new dataframes based on party. Hold all republicans in one dataframe and all democrats in the other. These will be our two different "samples."



In [199]:
### YOUR WORK HERE

column_headers = ['party','handicapped-infants','water-project',
                  'budget','physician-fee-freeze', 'el-salvador-aid',
                  'religious-groups','anti-satellite-ban',
                  'aid-to-contras','mx-missile','immigration',
                  'synfuels', 'education', 'right-to-sue','crime','duty-free',
                  'south-africa']

votes84 = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                      names=column_headers, na_values="?")

votes84 = votes84.replace({'y':1, 'n':0})

votes84rep = votes84[votes84['party']=='republican']
votes84dem = votes84[votes84['party']=='democrat']

### 2) Perform two sample T-tests on different issues and report the results.

- Find an issue that democrats support more than republicans with p < 0.01 (significant at the 99% level).
- Find an issue that republicans support more than democrats with p < 0.01 (significant at the 99% level).
- Find an issue where the difference between republicans and democrats has p > 0.1 (Not significant at the 90% level - i.e. there may not be much of a difference the two sample means)

Please for each test that you run state your null and alternative hypothesis and then write a conclusion reflecting on the null and alternative hypothesis.

Remember, that two-sample t-tests will only tell us if the two groups are *different* from one another. We'll have to look at their sample means directly or use the sign on the t-statistic to know which of the two sample means is larger. 

In [208]:
### YOUR WORK HERE

print(votes84rep['budget'].mean())
print(votes84dem['budget'].mean())

stats.ttest_ind(votes84rep['budget'], votes84dem['budget'], nan_policy='omit')

# The resulting pvalue is incredibly small - this is an issue that democrats
# support much more than republicans with a statistically significant 
# confidence value



0.13414634146341464
0.8884615384615384


Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [209]:
print(votes84rep['religious-groups'].mean())
print(votes84dem['religious-groups'].mean())

stats.ttest_ind(votes84rep['religious-groups'], votes84dem['religious-groups'], nan_policy='omit')

# The resulting pvalue is incredibly small - this is an issue that republicans
# support much more than democrats with a statistically significant 
# confidence value


0.8975903614457831
0.47674418604651164


Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

In [228]:
print(votes84rep['water-project'].mean())
print(votes84dem['water-project'].mean())

stats.ttest_ind(votes84rep['water-project'], votes84dem['water-project'], nan_policy='omit')

# The resulting pvalue is quite large with a confidence value of 7%
# This is not an issue that republicans and democrats disagree on with a 93%
# confidence value

0.5067567567567568
0.502092050209205


Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

## Stretch Goals:

### 1) Use functions and some iterator (for loop, .apply(), list comprehension, etc.) to perform two sample t-tests on every issue in the dataset in an automated fashion.

In [ ]:
### YOUR WORK HERE



### 2) Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### Be sure to check your work using Scipy!





In [ ]:
### YOUR WORK HERE

### 3) Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Be sure to check your work using Scipy!

In [ ]:
### YOUR WORK HERE